In [1]:
import csv
import re

import math
import numpy as np
import pandas as pd
from enum import Enum

import psycopg2
from sqlalchemy import create_engine
import neo4j

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# Database Setup

This section establishes connection with the Postgres and Neo4j databases, and exposes a set of utilities for interacting with these connections.

In [2]:
# Establish connection with postgresql database
postgres_connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [3]:
# Establish connection with neo4j database
neo4j_driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [4]:
class postgres_controller():
    """
    Controller class to manage interaction with a Postgres server.
    """
    def __init__(self, connection: psycopg2.extensions.connection):
        """
        Constructor for the postgres_controller class.

        Args:
            connection (psycopg2.extensions.connection): Connection object to the Postgres server.
        """
        self.connection = connection
        self.cursor = connection.cursor()
    
    def query(
            self, 
            query_string: str, 
            rollback_before: bool = True, 
            rollback_after: bool  = False, 
            commit: bool = True, 
            **query_params: any
        ) -> None:
        """
        Execute a query on the Postgres server.

        Args:
            query_string (str): Query string to execute.
            rollback_before (bool): Whether to rollback transaction before query execution.
            rollback_after (bool): Whether to rollback transaction after query execution.
            commit (bool): Whether to commit transaction after query execution.
            query_params (dict): Parameters to be passed to the query.
        """
        
        if rollback_before:
            self.connection.rollback()
        
        # Execute the query
        self.cursor.execute(query_string, list(query_params.values()))
        
        if rollback_after:
            self.connection.rollback()
            
        if commit:
            # Commit the transaction to the database
            self.connection.commit()
    
    def query_to_df(
            self, 
            query_string: str, 
            rollback_before: bool = True, 
            rollback_after: bool  = False, 
            commit: bool = True, 
            *query_params: any
        ) -> pd.DataFrame:
        """
        Execute a query on the Postgres server and return the result as a pandas DataFrame.
        
        Args:
            query_string (str): Query string to execute.
            rollback_before (bool): Whether to rollback transaction before query execution.
            rollback_after (bool): Whether to rollback transaction after query execution.
            commit (bool): Whether to commit transaction after query execution.
            query_params (dict): Parameters to be passed to the query.

        Returns:
            pd.DataFrame: Result of the query as a pandas DataFrame.
        """
        
        if rollback_before:
            self.connection.rollback()

        # Execute the query and write the result to a pandas DataFrame 
        response = pd.read_sql_query(query_string, self.connection, params=query_params)
        
        if rollback_after:
            self.connection.rollback()
            
        if commit:
            # Commit the transaction to the database
            self.connection.commit()
            
        return response
    

In [5]:
Direction = Enum("Direction", ["ONE_WAY", "TWO_WAY"])
NodeType = Enum("NodeType", ["DEFAULT_NODE", "HOSPITAL", "INSURANCE"])
RelationshipType = Enum("RelationshipType", ["DEFAULT_RELATIONSHIP", "CHARGED", "PAYED"])

class neo4j_controller():
    """
    Controller class to manage interaction with a Neo4j server.
    """
    def __init__(self, driver: neo4j.GraphDatabase.driver):
        """
        Constructor for the neo4j_controller class.

        Args:
            driver (neo4j.GraphDatabase.driver): Driver object to the Neo4j server
        """
        self.driver = driver
        self.session = driver.session(database="neo4j")
    
    def query(self, query_string: str, **kwargs) -> neo4j.Result:
        """
        Execute a query on the Neo4j server (automatically commits the transaction).

        Args:
            query_string (str): Query string to execute.
            **kwargs: Keyword arguments to be passed with the query.

        Returns:
            neo4j.Result: Result of the query.
        """
        return self.session.run(query_string, **kwargs)
    
    def query_to_pandas(self, query_string: str) -> pd.DataFrame:
        """
        Execute a query on the Neo4j server and return the result as a pandas DataFrame.

        Args:
            query_string (str): Query string to execute.

        Returns:
            pd.DataFrame: Result of the query as a pandas DataFrame.
        """
        response = self.query(query_string)
        return pd.DataFrame([result.values() for result in response], columns=response.keys())
    
    def clear_database(self):
        """
        Clear the entire database (removes all nodes and relationships).
        """
        # Remove all nodes with relationships
        self.query("match (node)-[relationship]->() delete node, relationship")

        # Remove all nodes without relationships
        self.query("match (node) delete node")
        
    def create_node(self, node_type: NodeType, properties: dict[str, any]) -> neo4j.Result:
        """
        Create a node in the Neo4j database.
        
        Args:
            node_type (NodeType): Type of node to create.
            properties (dict[str, any]): Properties to assign to the node.

        Returns:
            neo4j.Result: Result of the query.
        """

        query_string = f"CREATE (:{node_type.name} $properties)"
        
        self.query(query_string, properties=properties)
        
    
    def create_relationship(
            self, 
            node1_name: str, 
            node2_name: str, 
            node1_type: NodeType = NodeType.DEFAULT_NODE,  
            node2_type: NodeType = NodeType.DEFAULT_NODE,
            relationship_type: RelationshipType = RelationshipType.DEFAULT_RELATIONSHIP,
            properties: dict[str, any] = {}, 
            direction: Direction = Direction.ONE_WAY
        ):
        """
        Create a relationship between a two nodes in the Neo4j database.

        Args:
        """

        query_string = f"match (node1:{node1_type.name}), (node2:{node2_type.name}) where node1.name = $node1_name and node2.name = $node2_name create(node1)-[:{relationship_type.name} $properties]->(node2)"

        if direction == Direction.TWO_WAY:
            # Update the query to create the same relationship in the other direction
            query_string += f",create(node2)-[:{relationship_type.name} $properties]->(node1)"

        self.query(query_string, node1_name=node1_name, node2_name=node2_name, properties=properties)



# Data Wrangling

In [6]:
# Enhanced preprocessing function
def preprocess_plan_name(name):
    # Convert to lowercase
    name = name.lower()
    
    # Replace underscores and special characters
    name = re.sub(r'[_\-.]', ' ', name)
    
    # Extract meaningful parts (numbers, letters, spaces)
    name = re.sub(r'[^a-z0-9\s]', '', name)
    
    # Remove extra spaces
    name = ' '.join(name.split())
    
    return name

# Add feature extraction functions
def extract_features(plan_name):
    features = {}
    
    # Extract numeric values
    features['has_numbers'] = int(bool(re.search(r'\d', plan_name)))
    
    # Check for common keywords
    keywords = ['hmo', 'ppo', 'pos', 'medicare', 'medicaid', 'advantage', 'choice']
    for keyword in keywords:
        features[f'has_{keyword}'] = int(keyword in plan_name)
    
    return features

# Create feature DataFrame
def create_feature_matrix(plan_names):
    feature_dicts = [extract_features(name) for name in plan_names]
    return pd.DataFrame(feature_dicts)

In [7]:
# Function to predict new insurance plans
def predict_plan_classification(plan_name):
    # Preprocess the plan name
    processed_name = preprocess_plan_name(plan_name)
    
    # Create features
    text_features = vectorizer.transform([processed_name])
    additional_features = create_feature_matrix([processed_name]).values
    combined_features = np.hstack([text_features.toarray(), additional_features])
    
    # Make predictions
    carrier_pred = le_carrier.inverse_transform(carrier_model.predict(combined_features))
    plan_type_pred = le_plan_type.inverse_transform(plan_type_model.predict(combined_features))
    
    return {
        'plan_name': plan_name,
        'predicted_carrier': carrier_pred[0],
        'predicted_plan_type': plan_type_pred[0],
        'carrier_probability': np.max(carrier_model.predict_proba(combined_features)),
        'plan_type_probability': np.max(plan_type_model.predict_proba(combined_features))
    }

# Batch prediction function
def batch_predict(unlabeled_data):
    cleaned_names = unlabeled_data.payer_name.apply(preprocess_plan_name)
    text_features = vectorizer.transform(cleaned_names)
    additional_features = create_feature_matrix(cleaned_names).values
    combined_features = np.hstack([text_features.toarray(), additional_features])
    
    carrier_preds = le_carrier.inverse_transform(carrier_model.predict(combined_features))
    plan_type_preds = le_plan_type.inverse_transform(plan_type_model.predict(combined_features))
    
    carrier_probs = np.max(carrier_model.predict_proba(combined_features), axis=1)
    plan_type_probs = np.max(plan_type_model.predict_proba(combined_features), axis=1)
    
    return pd.DataFrame({
        'plan': unlabeled_data.payer_name,
        'predicted_carrier': carrier_preds,
        'predicted_plan_type': plan_type_preds,
        'carrier_confidence': carrier_probs,
        'plan_type_confidence': plan_type_probs
    })


In [8]:
INSURANCE_TRAINING_DATA = "/user/projects/project-3-TylerKoon/data/insurance_training_data.csv"

# Create DataFrame
df = pd.read_csv(INSURANCE_TRAINING_DATA)

# Preprocess plan names
df['processed_name'] = df['plan'].apply(preprocess_plan_name)

# Create additional features
feature_df = create_feature_matrix(df['processed_name'])
df = pd.concat([df, feature_df], axis=1)

# Convert categorical variables to numerical
le_carrier = LabelEncoder()
le_plan_type = LabelEncoder()

df['carrier_encoded'] = le_carrier.fit_transform(df['carrier'])
df['plan_type_encoded'] = le_plan_type.fit_transform(df['type'])

# Create vectorizer for plan names with improved settings
vectorizer = TfidfVectorizer(
    analyzer='char_wb',
    ngram_range=(2, 5),
    max_features=1000,
    min_df=2
)

# Prepare features
X_text = vectorizer.fit_transform(df['processed_name'])
X_additional = feature_df.values
X = np.hstack([X_text.toarray(), X_additional])

# Prepare target variables
y_carrier = df['carrier_encoded']
y_plan_type = df['plan_type_encoded']

# Split the data
X_train_carrier, X_test_carrier, y_train_carrier, y_test_carrier = train_test_split(
    X, y_carrier, test_size=0.2, random_state=42, stratify=y_carrier
)

X_train_plan, X_test_plan, y_train_plan, y_test_plan = train_test_split(
    X, y_plan_type, test_size=0.2, random_state=42, stratify=y_plan_type
)

In [9]:
# Define model parameters for grid search
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Train models with grid search
carrier_model = GridSearchCV(
    RandomForestClassifier(random_state=42),
    rf_params,
    cv=5,
    n_jobs=-1,
    verbose=1
)

plan_type_model = GridSearchCV(
    RandomForestClassifier(random_state=42),
    rf_params,
    cv=5,
    n_jobs=-1,
    verbose=1
)

In [10]:
# Fit models
carrier_model.fit(X_train_carrier, y_train_carrier)
plan_type_model.fit(X_train_plan, y_train_plan)

# Make predictions
carrier_predictions = carrier_model.predict(X_test_carrier)
plan_type_predictions = plan_type_model.predict(X_test_plan)

# Print model performance
print("\nCarrier Model Performance:")
print(classification_report(y_test_carrier, carrier_predictions))
print("\nPlan Type Model Performance:")
print(classification_report(y_test_plan, plan_type_predictions))


Fitting 5 folds for each of 24 candidates, totalling 120 fits


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Fitting 5 folds for each of 24 candidates, totalling 120 fits


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_split.py:666: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"



Carrier Model Performance:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         1
           5       1.00      0.75      0.86         8
           7       1.00      1.00      1.00         4
           9       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         0
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         3
          16       0.67      1.00      0.80         2
          18       1.00      1.00      1.00         4
          19       0.00      0.00      0.00         1
          21       0.00      0.00      0.00         2

    accuracy                           0.80        30
   macro avg       0.67      0.67      0.67        30
weighted avg       0.84      0.80      0.82        30


Plan Type Model Performance:
              precisi

/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1

In [11]:
RATE_CSV_PATH = "/user/projects/project-3-TylerKoon/data/illinois_rates.csv"

# 
rates = pd.read_csv(RATE_CSV_PATH)

# Predict carrier and plan type for each payer
predictided_plans = batch_predict(rates)
predictided_plans.head()

# Add predicitons
rates.insert(rates.columns.get_loc("payer_name") + 1, "carrier", predictided_plans["predicted_carrier"])
rates.insert(rates.columns.get_loc("payer_name") + 2, "plan_type", predictided_plans["predicted_plan_type"])

# Write the updates rates csv to disk
rates.to_csv("/user/projects/project-3-TylerKoon/data/illinois_rates_updated.csv", index=False)

# Relational Database Configuration

In [12]:
# Create an instance of the postgres_controller to interact with the Postgres server
postgres_controller = postgres_controller(postgres_connection)

In [13]:
query_string = """
drop table if exists hospital cascade;
"""

# Drop the `hospital` table if it already exists
postgres_controller.query(query_string)

In [14]:
query_string = """
create table hospital (
  id varchar(6) NOT NULL,
  ein varchar(11),
  name varchar(100),
  alt_name varchar(100),
  system_name varchar(100),
  addr varchar(100),
  city varchar(50),
  state char(2),
  zip char(5),
  phone varchar(14),
  urban_rural char(1),
  category varchar(50),
  control_type varchar(50),
  medicare_termination_status varchar(100),
  last_updated date,
  file_name varchar(1000),
  mrf_url varchar(2048),
  permalink varchar(2048),
  transparency_page varchar(2048),
  additional_notes text,
  primary key (id)
)
"""

# Create the `hospital` table
postgres_controller.query(query_string)

In [15]:
query_string = """
drop table if exists rate;
"""

# Drop the `rate` table if it already exists
postgres_controller.query(query_string)

In [16]:
# Define a set of enum types used in the `rate` table schema
type_declarations = [
    "drug_measurement_type as enum ('gr','mg','ml','un')",
    "billing_class_type as enum ('professional','facility')",
    "setting_type as enum ('inpatient','outpatient','both')",
    "rate_category_type as enum ('gross','cash','min','max','negotiated')",
    "contracting_method_type as enum ('capitation','case rate','fee schedule','percent of total billed charge','per diem','other')",
]

for type_declaration in type_declarations:
    # Drop the type if it already exists
    postgres_controller.query(f"drop type if exists {type_declaration.split(' ')[0]}")
    
    # Create the type
    postgres_controller.query(f"create type {type_declaration}")


In [17]:
query_string = """
create table rate (
  hospital_id varchar(6) not null,
  row_id numeric not null default '0',
  line_type varchar(40),
  description varchar(2000),
  rev_code varchar(4),
  local_code varchar(40),
  code varchar(50),
  ms_drg varchar(3),
  apr_drg varchar(5),
  eapg varchar(5),
  hcpcs_cpt varchar(5),
  modifiers varchar(14),
  alt_hcpcs_cpt varchar(5),
  thru varchar(10),
  apc varchar(4),
  icd varchar(15),
  ndc varchar(13),
  drug_hcpcs_multiplier int,
  drug_quantity varchar(25),
  drug_unit_of_measurement varchar(25),
  drug_type_of_measurement drug_measurement_type,
  billing_class billing_class_type,
  setting setting_type,
  rate_category rate_category_type not null,
  payer_name varchar(1000),
  carrier varchar(1000),
  plan_type varchar(1000),
  plan_name varchar(1000),
  standard_charge decimal(14,2),
  standard_charge_percent decimal(6,2),
  contracting_method contracting_method_Type,
  additional_generic_notes text,
  additional_payer_specific_notes text,
  primary key (hospital_id, row_id),
  constraint ktqb0v0a foreign key (hospital_id) references hospital (id)
)
"""

# Create the `rate` table
postgres_controller.query(query_string)

In [18]:
HOSPITAL_CSV_PATH = "/user/projects/project-3-TylerKoon/data/illinois_hospitals.csv"

query_string = """
copy hospital
from %s delimiter ',' csv header;
"""

# Load the hospital data into the `hospital` table
postgres_controller.query(query_string, file_path=HOSPITAL_CSV_PATH)

In [19]:
UPDATED_RATE_CSV_PATH = "/user/projects/project-3-TylerKoon/data/illinois_rates_updated.csv"

query_string = """
copy rate
from %s delimiter ',' csv header;
"""

# Load the rate data into the `rate` table
postgres_controller.query(query_string, file_path=UPDATED_RATE_CSV_PATH)

# Graph Database Configuration

In [20]:
# Create an instance of the neo4j_controller to interact with the Neo4j server
neo4j_controller = neo4j_controller(neo4j_driver)

In [21]:
# Clear the Neo4j database to start fresh
neo4j_controller.clear_database()

In [ ]:
# # Select all hospitals from the `hospital` table
# hospitals = postgres_controller.query_to_df("select * from hospital")

# for row_id, hospital in hospitals.iterrows():
#     # Create a node for each hospital
#     neo4j_controller.create_node(NodeType.HOSPITAL, {"name": hospital["name"], "id": hospital["id"]})

In [ ]:
# # Select all unique insurances from the `rate` table
# insurances = postgres_controller.query_to_df("select distinct carrier, plan_type from rate")

# for row_id, insurance in insurances.iterrows():
#     # Create a node for each insurance
#     neo4j_controller.create_node(NodeType.INSURANCE, {"name": insurance["carrier"], "plan_type": insurance["plan_type"]})

In [ ]:
# query_string = """
# select hospital.id as "hospital_id", 
#        hospital.name as "hospital_name",
#        rate.code as "drg_code",
#        rate.carrier,
#        rate.plan_type,
#        rate.standard_charge
# from rate
# join hospital on rate.hospital_id = hospital.id
# """

# # Select all charges from the `rate` table
# rates = postgres_controller.query_to_df(query_string)

# for row_id, rate in rates.iterrows():    
#     # Create a relationship between the hospital and the insurance
#     neo4j_controller.create_relationship(
#         rate["hospital_name"], # Name of the first node 
#         rate["carrier"], # Name of the second node
#         NodeType.HOSPITAL, # Type of the first node
#         NodeType.INSURANCE, # Type of the second node
#         RelationshipType.CHARGED, # Type of the relationship
#         {"drg_code": rate["drg_code"], "charge": rate["standard_charge"]} # Properties attached to the relationship
# )

In [22]:
query_string = """
select hospital.id as "hospital_id", 
       hospital.name as "hospital_name",
       rate.code as "drg_code",
       rate.carrier as "insurance_carrier",
       rate.standard_charge
from rate
INNER join hospital on rate.hospital_id = hospital.id
"""

# Select all charges from the `rate` table
charges = postgres_controller.query_to_df(query_string)


In [23]:
# Create a node for each hospital
for row_id, hospital_record in charges[['hospital_id', 'hospital_name']].drop_duplicates().reset_index(drop=True).iterrows():
    neo4j_controller.create_node(NodeType.HOSPITAL, {"name": hospital_record['hospital_name'], "id":hospital_record['hospital_id']})

In [24]:
# Create a node for each insurance
for row_id, hospital_record in charges[['insurance_carrier']].drop_duplicates().reset_index(drop=True).iterrows():
    neo4j_controller.create_node(NodeType.INSURANCE, {"name": hospital_record["insurance_carrier"]})

In [25]:
# Create relationships
for row_id, charge in charges.iterrows():
    # Create a relationship between the hospital and the insurance
    neo4j_controller.create_relationship(
        charge["hospital_name"], # Name of the first node 
        charge["insurance_carrier"], # Name of the second node
        NodeType.HOSPITAL, # Type of the first node
        NodeType.INSURANCE, # Type of the second node
        RelationshipType.CHARGED, # Type of the relationship
        {"drg_code": charge["drg_code"], "charge": charge["standard_charge"]} # Properties attached to the relationship
)

# Analysis
This section covers our initial exploration of these data. 

In [26]:
# Look at the average charge for each hospital and insurance carrier
charges.groupby(["hospital_name", "insurance_carrier"]).agg('mean').reset_index()

,hospital_name,insurance_carrier,standard_charge
0,ALEXIAN BROTHERS MEDICAL CENTER 1,Aetna,41495.906667
1,ALEXIAN BROTHERS MEDICAL CENTER 1,Ambetter,57579.520000
2,ALEXIAN BROTHERS MEDICAL CENTER 1,Blue Cross,45967.028889
3,ALEXIAN BROTHERS MEDICAL CENTER 1,Cigna,43974.460000
4,ALEXIAN BROTHERS MEDICAL CENTER 1,Health Alliance,44428.640000
...,...,...,...
110,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,Golden Rule/UnitedHealthcare,63071.000000
111,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,Healthlink,61467.500000
112,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,Humana,50741.871111
113,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,UnitedHealthcare,63071.000000


# Algorithms
This section covers the application of graph algorithms to these data, and the accompanying analyses/exploration.

### Degree Centrality
The motivation behind this algorithm is to identify well connected/supported insurances in the area. Using this information, we could ensure that we develop strong relationships with these insurers to better support the wider community. We might also be able to target less supported insurance plans so that we can increase network coverage for those patients that have relatively small networks in the area.

In [27]:
# Setup a projection
# Note: The `MATCH` statement is used to define what the graph will look like, 
# and the `RETURN` statement is used to run the `gds.graph.project` procedure
# and create and return the graph projection. Notice the first argument to the
# procedure is the graph name "dc_graph". This will be used to refer to the graph
# in future queries.
query = """
    MATCH (source:HOSPITAL)-[r:CHARGED]->(target:INSURANCE)
    RETURN gds.graph.project(
        'dc_graph',
        target,
        source,
        { relationshipProperties: r { .charge } }
    )
    """

neo4j_controller.query(query) 

In [28]:
# Run the Degree Centrality algorithm on our projection
# Note: Just like with the graph projection, we call a procedure 
# (in this case `gds.degree.stream`) and pass in arguments (the graph name "dc_graph").
# We also use the YIELD keyword for creating a reference to values returned by the 
# `gds.degree.stream` procedure that we are interested in using/returning. 
query = """
    CALL gds.degree.stream('dc_graph') 
    YIELD nodeId, score 
    RETURN gds.util.asNode(nodeId).name AS name, score ORDER BY score DESC
"""

degree_centrality_df = neo4j_controller.query_to_pandas(query)
degree_centrality_df

,name,score
0,Blue Cross,196.0
1,UnitedHealthcare,106.0
2,Cigna,91.0
3,Other,74.0
4,Humana,67.0
5,Medicare,27.0
6,Aetna,22.0
7,Healthlink,22.0
8,Health Alliance,18.0
9,Golden Rule/UnitedHealthcare,7.0


### Identifying Similar Insurance Carriers
The motivation behind this algorithm is to identify similar insurance plans with the goal of improving pricing consistancy---ensuring prices are consistent with other hospitals in the area, but also that prices are consistent between insurance providers/plans that are similar.

We could also consider deploying this in a consumer facing application so that users can compare plans which offer similar coverage but might have different prices. This could be a valuable tool when selecting an insurance provider/plan (more transparency around how a provider/plan compares to competitors). This information might also be valuable when selecting a plan from a prescribed list of available plans (for example, those offered b employers). 

In [29]:
# Setup a projection
query = """
    MATCH (source:INSURANCE)
    OPTIONAL MATCH (target:HOSPITAL)-[r:CHARGED]->(source)
    RETURN gds.graph.project(
        'si_graph',
        source,
        target,
        { relationshipProperties: r { .charge } }
    )
    """

neo4j_controller.query(query) 

In [30]:
query = """
    CALL gds.nodeSimilarity.stream('si_graph') 
    YIELD node1, node2, similarity 
    RETURN gds.util.asNode(node1).name AS carrier1, gds.util.asNode(node2).name AS carrier2, similarity 
    ORDER BY similarity DESC
"""

node_similarity_df = neo4j_controller.query_to_pandas(query)
node_similarity_df

,carrier1,carrier2,similarity
0,Blue Cross,Cigna,1.000000
1,Blue Cross,Humana,1.000000
2,Blue Cross,UnitedHealthcare,1.000000
3,UnitedHealthcare,Cigna,1.000000
4,UnitedHealthcare,Humana,1.000000
...,...,...,...
215,Ascension,Humana,0.090909
216,Wellcare,Humana,0.090909
217,AARP,Humana,0.090909
218,Amita Health,Humana,0.090909


In [31]:
query = """
    CALL gds.nodeSimilarity.write('si_graph', {writeRelationshipType: 'SIMILAR', writeProperty: 'similarity_score'}) 
    YIELD nodesCompared, relationshipsWritten
"""

node_similarity_df = neo4j_controller.query_to_pandas(query)
node_similarity_df

,nodesCompared,relationshipsWritten
0,22,220


In [32]:
query = """
CALL gds.graph.project(
    'hospital_charges',
    ['HOSPITAL', 'INSURANCE'],
    {
        CHARGED: {
            properties: 'charge',
            orientation: 'UNDIRECTED'
        }
    }
)
"""
neo4j_controller.query(query)

In [34]:
# Run node similarity algorithm again, but this time on the hospital_charges graph
query = """
MATCH (h1:HOSPITAL)-[r1:CHARGED]->(i:INSURANCE)
MATCH (h2:HOSPITAL)-[r2:CHARGED]->(i:INSURANCE)
WHERE h1 <> h2
RETURN h1.name AS Hospital1, 
       h2.name AS Hospital2,
       1 - abs(avg(r1.charge - r2.charge)/avg(r1.charge)) as ChargeSimilarity
ORDER BY ChargeSimilarity DESC
LIMIT 10
"""
charge_similarity_df = neo4j_controller.query_to_pandas(query)
charge_similarity_df

,Hospital1,Hospital2,ChargeSimilarity
0,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,PRESENCE SAINTS MARY AND ELIZABETH MEDICAL CENTER,1.000000
1,PRESENCE SAINTS MARY AND ELIZABETH MEDICAL CENTER,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,1.000000
2,PRESENCE ST MARYS HOSPITAL,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,0.999476
3,PRESENCE ST MARYS HOSPITAL,PRESENCE SAINTS MARY AND ELIZABETH MEDICAL CENTER,0.999476
4,PRESENCE SAINTS MARY AND ELIZABETH MEDICAL CENTER,PRESENCE ST MARYS HOSPITAL,0.999475
5,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,PRESENCE ST MARYS HOSPITAL,0.999475
6,PRESENCE ST MARYS HOSPITAL,PRESENCE SAINT JOSEPH HOSPITAL - CHICAGO,0.998936
7,PRESENCE SAINT JOSEPH HOSPITAL - CHICAGO,PRESENCE ST MARYS HOSPITAL,0.998935
8,PRESENCE RESURRECTION MEDICAL CENTER,PRESENCE SAINT JOSEPH HOSPITAL - ELGIN,0.998918
9,PRESENCE SAINT JOSEPH HOSPITAL - ELGIN,PRESENCE RESURRECTION MEDICAL CENTER,0.998917


### Community Detection (Louvain)
The motivation behind this algorithm is to try and cluster insurance providers with hospitals. With analyses of these clusters, we might be able to extract valuable information that could be used to gain an edge over competitors (such as communities are being overcharged for a procedure, which communities are lacking adequate coverage of insurance providers, etc.)

In [35]:
# First, create the projection
query = """
CALL gds.graph.project(
    'hospital_insurance_network',
    ['HOSPITAL', 'INSURANCE'],
    {
        CHARGED: {
            orientation: 'UNDIRECTED'
        }
    }
)
"""
neo4j_controller.query(query)

In [36]:
# Run Louvain
query = """
CALL gds.louvain.stream('hospital_insurance_network')
YIELD nodeId, communityId
WITH gds.util.asNode(nodeId) as node, communityId
RETURN 
    CASE WHEN node:HOSPITAL THEN 'Hospital' ELSE 'Insurance' END as Type,
    node.name as Name,
    communityId as Community
ORDER BY communityId, Type, Name
"""
louvain_df = neo4j_controller.query_to_pandas(query)
louvain_df

,Type,Name,Community
0,Hospital,PRESENCE MERCY MEDICAL CENTER,12
1,Hospital,PRESENCE SAINT JOSEPH HOSPITAL - ELGIN,12
2,Hospital,PRESENCE SAINT JOSEPH MEDICAL CENTER,12
3,Hospital,PRESENCE ST MARYS HOSPITAL,12
4,Insurance,Blue Cross,12
5,Insurance,Caterpillar,12
6,Insurance,Coventry,12
7,Insurance,Great West,12
8,Insurance,Health Alliance,12
9,Insurance,Self Pay,12


In [37]:
# Create a new projection
query = """
CALL gds.graph.project(
    'healthcare_graph_1',
    ['HOSPITAL', 'INSURANCE'],
    ['CHARGED']
)
"""
neo4j_controller.query(query)

In [41]:
# Louvain Community Detection for HOSPITAL -> INSURANCE
query = """
    CALL gds.louvain.stream('dc_graph') 
    YIELD nodeId, communityId 
    RETURN gds.util.asNode(nodeId).name AS name, communityId ORDER BY communityId
"""
louvain_df = neo4j_controller.query_to_pandas(query)
louvain_df

,name,communityId
0,Blue Cross,32
1,PRESENCE SAINT JOSEPH MEDICAL CENTER,32
2,Cigna,32
3,Self Pay,32
4,Coventry,32
5,Caterpillar,32
6,Health Alliance,32
7,Great West,32
8,Blue Cross,32
9,ST MARY & ELIZABETH MED CTR-CLAREMONT CAMPUS,32


### PageRank
The goal with this algorithm is similar to that of degree centrality: identify influential and well connected insurance providers so that we can better accommodate existing patients (and perhaps even target those patients which have historically had poor network coverage in the area).

In [39]:
# First, create the projection
query = """
CALL gds.graph.project(
    'healthcare_graph',
    ['HOSPITAL', 'INSURANCE'],
    ['CHARGED']
)
"""
neo4j_controller.query(query)

In [40]:
# Run Louvain
query = """
CALL gds.pageRank.stream('healthcare_graph')
YIELD nodeId, score
MATCH (n) WHERE id(n) = nodeId
RETURN n.name as Name, labels(n) as Type, score as PageRankScore
ORDER BY PageRankScore DESC
LIMIT 10
"""
communities_df = neo4j_controller.query_to_pandas(query)
communities_df

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated function: `id`.} {position: line: 4, column: 17, offset: 82} for query: "\nCALL gds.pageRank.stream('healthcare_graph')\nYIELD nodeId, score\nMATCH (n) WHERE id(n) = nodeId\nRETURN n.name as Name, labels(n) as Type, score as PageRankScore\nORDER BY PageRankScore DESC\nLIMIT 10\n"


,Name,Type,PageRankScore
0,Blue Cross,[INSURANCE],0.558781
1,UnitedHealthcare,[INSURANCE],0.366083
2,Other,[INSURANCE],0.364395
3,Cigna,[INSURANCE],0.337618
4,Humana,[INSURANCE],0.290281
5,Healthlink,[INSURANCE],0.255214
6,Medicare,[INSURANCE],0.213773
7,Aetna,[INSURANCE],0.199864
8,Health Alliance,[INSURANCE],0.190262
9,Golden Rule/UnitedHealthcare,[INSURANCE],0.164455
